In [1]:
## Definition of the DrugSDA SCP server, including basic operations such as connect, disconnect, list_tools, and parse_result.
import asyncio
import json
from mcp.client.streamable_http import streamablehttp_client
from mcp import ClientSession

DrugSDA_Tool_SERVER_URL = "https://scp.intern-ai.org.cn/api/v1/mcp/2/DrugSDA-Tool"      ## DrugSDA-Tool Server

class DrugSDAClient:    
    def __init__(self, server_url: str):
        self.server_url = server_url
        self.session = None
        
    async def connect(self):
        print(f"server url: {self.server_url}")
        try:
            self.transport = streamablehttp_client(
                url=self.server_url,
                headers={"SCP-HUB-API-KEY": "sk-a0033dde-b3cd-413b-adbe-980bc78d6126"}
            )
            self.read, self.write, self.get_session_id = await self.transport.__aenter__()
            
            self.session_ctx = ClientSession(self.read, self.write)
            self.session = await self.session_ctx.__aenter__()

            await self.session.initialize()
            session_id = self.get_session_id()
            
            print(f"✓ connect success")
            return True
            
        except Exception as e:
            print(f"✗ connect failure: {e}")
            import traceback
            traceback.print_exc()
            return False
    
    async def disconnect(self):
        try:
            if self.session:
                await self.session_ctx.__aexit__(None, None, None)
            if hasattr(self, 'transport'):
                await self.transport.__aexit__(None, None, None)
            print("✓ already disconnect")
        except Exception as e:
            print(f"✗ disconnect error: {e}")
    
    async def list_tools(self):        
        try:
            tools_list = await self.session.list_tools()
            print(f"tool count: {len(tools_list.tools)}")
            
            for i, tool in enumerate(tools_list.tools, 1):
                print(f"{i:2d}. {tool.name}")
                if tool.description:
                    desc_line = tool.description.split('\n')[0]
                    print(f"    {desc_line}")
            
            print(f"✓ Get tool list success")
            return tools_list.tools
            
        except Exception as e:
            print(f"✗ Get tool list fail: {e}")
            return []
    
    def parse_result(self, result):
        try:
            if hasattr(result, 'content') and result.content:
                content = result.content[0]
                if hasattr(content, 'text'):
                    return json.loads(content.text)
            return str(result)
        except Exception as e:
            return {"error": f"parse error: {e}", "raw": str(result)}

In [ ]:
## Fluorescent protein retrieval and mutation prediction
from openai import OpenAI
import json

def call_llm_stream(prompt: str, system_prompt: str = "You are a helpful assistant") -> str:
    client = OpenAI(
        base_url="****",
        api_key="****"
    )
    
    response = client.chat.completions.create(
        model="deepseek-reasoner",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=0.8
    )
    
    return response.choices[0].message.content

async def main(sequence):
    client = DrugSDAClient(DrugSDA_Tool_SERVER_URL)
    if not await client.connect():
        print("connection failed")
        return
    
    ## step 2. Protein structure prediction, call SCP tool pred_protein_structure_esmfold
    result = await client.session.call_tool(
        "pred_protein_structure_esmfold",
        arguments={
            "sequence": sequence
        }
    )
    
    result_data = client.parse_result(result)
    print (result_data)
    pdb_path = result_data["pdb_path"]
    print ("protein structure: ", pdb_path)
    
    ## step 3. Mutant sequence prediction, call SCP tool pred_mutant_sequence
    result = await client.session.call_tool(
        "pred_mutant_sequence",
        arguments={
            "sequence": sequence,
            "pdb_file_path": pdb_path,
            "top_k": 10
        }
    )
    
    result_data = client.parse_result(result)
    for item in result_data["mutated_sequences"]:
        print (item)
    
    await client.disconnect()

if __name__ == '__main__':
    
    ## step 1. paper search for candidate fluorescent proteins, call LLM deepseek-reasoner
    prompt = "Task: The experiment aims to enhance the fluorescence intensity of eGFP through protein mutation. Please retrieve one candidate protein sequence that meets the following criteria."
    system_prompt = f"""Requirements:
1. Output only a list, with no additional content. Each element in the list must be in the format: {{"Name": "", "Mutated Amino Acid Sequence": "", "Reference": ""}}.
2. The cited reference must be a recently published English-language paper (must be real and from a journal with an impact factor greater than 10).
3. The mutated amino acid sequence provided must correspond to a real, experimentally validated variant."""
    response = call_llm_stream(prompt, system_prompt)
    protein_seq = json.loads(response)[0]["Mutated Amino Acid Sequence"]
    print ("protein sequence: ", protein_seq)
    
    await main(protein_seq)

protein sequence:  MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYITADKQKNGIKANFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITLGMDELYK
server url: https://scp.intern-ai.org.cn/api/v1/mcp/3/DrugSDA-Model
✓ connect success
{'status': 'success', 'msg': 'Protein structure saved successfully at exp_data/pdb_9406.pdb', 'pdb_path': '/root/lwj/wll/code/DrugAgentTools/exp_data/pdb_9406.pdb'}
protein structure:  /root/lwj/wll/code/DrugAgentTools/exp_data/pdb_9406.pdb
MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVVRIELKGIDFKEDGNILGHKLEYNYNSHNVYITADKQKNGIKANFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITLGMDELYK
MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDGGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYITADKQKNGIKANFKIRHNIEDGSV